<a href="https://colab.research.google.com/github/DinDev3/Python-Machine-Learning/blob/master/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

Install packages
- pip install sklearn
- pip install pandas
- pip install quandl

---
Stock Prices


In [0]:
# !pip --version
# !pip install --upgrade pip
!pip install sklearn
!pip install pandas
!pip install quandl

In [0]:
import pandas as pd
import quandl

In [20]:
# storing the tabled data received from quandl in a pandas dataframe
df = quandl.get("WIKI/GOOGL")       # https://www.quandl.com/

print(df.head())

              Open    High     Low  ...   Adj. Low  Adj. Close  Adj. Volume
Date                                ...                                    
2004-08-19  100.01  104.06   95.96  ...  48.128568   50.322842   44659000.0
2004-08-20  101.01  109.08  100.50  ...  50.405597   54.322689   22834300.0
2004-08-23  110.76  113.48  109.05  ...  54.693835   54.869377   18256100.0
2004-08-24  111.24  111.60  103.57  ...  51.945350   52.597363   15247300.0
2004-08-25  104.76  108.00  103.88  ...  52.100830   53.164113    9188600.0

[5 rows x 12 columns]


Each column is a feature here.

All these columns aren't needed to find a pattern.

Relationships between features can be identified in Deep learning, but not in Regression.

We want to keep only meaningful features.

In [25]:
# creating a long list of columns that we want to have
# recreating the dataframe to contain only the mentioned columns
df = df[['Adj. Open', 'Adj. High','Adj. Low','Adj. Close','Adj. Volume']]

# print(df.head())

# defining special relationships, to use as features
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Close'] * 100
df['PCT_Change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100

# creating a new dataframe with the special features
df = df[['Adj. Close','HL_PCT','PCT_Change','Adj. Volume']]

print(df.head())


            Adj. Close    HL_PCT  PCT_Change  Adj. Volume
Date                                                     
2004-08-19   50.322842  8.072956    0.324968   44659000.0
2004-08-20   54.322689  7.921706    7.227007   22834300.0
2004-08-23   54.869377  4.049360   -1.227880   18256100.0
2004-08-24   52.597363  7.657099   -5.726357   15247300.0
2004-08-25   53.164113  3.886792    1.183658    9188600.0


Features are kind of like attributes that make up the label.
The label is a prediction